In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyclim_noresm.general_util_funcs import global_avg
import matplotlib as mpl
from workflow.scripts.plotting_tools import global_map
from workflow.scripts.utils import compute_annual_emission_budget
import xesmf as xe
import cf_xarray as cfxr
import numpy as np

In [3]:
paths = snakemake.input.paths
grid_area = snakemake.input.areacello
common_grid = 'workflow/input_data/common_grid.nc'

In [4]:
dsets = []
budgets = {}
# ds_out = xe.util.grid_global(3, 2,cf=True)
for path, ga_path in zip(paths, grid_area):
    ds = xr.open_dataset(path)
    if ds.attrs.get('regrid_method'):
        ga = xr.open_dataset(common_grid)
    else:
        ga = xr.open_dataset(ga_path)
    
#     yearly_budget = ds[ds.variable_id].copy().resample(time='Y').sum()
    
#     yearly_budget = yearly_budget.mean(dim='time')*365*24*60*60
#     yearly_budget = yearly_budget*ga['cell_area'].load()
#     yearly_budget = yearly_budget.sum(dim=['lon','lat']).values
    budgets[ds.source_id],_=compute_annual_emission_budget(ds,ga)
    ds = ds.resample(time='Y').sum(keep_attrs=True).mean(dim='time',keep_attrs=True)
    ds = ds.assign({ds.variable_id:ds[ds.variable_id]*365*24*60*60}) 
#     ds = ds.groupby('time.month').mean(dim='time', keep_attrs=True).mean('month', keep_attrs=True)
#     regridder= xe.Regridder(ds,ds_out,'conservative')
    
    #ds = ds.rename({ds.variable_id:ds.source_id})
#     ds = regridder(ds,keep_attrs=True)
#     ds[ds.variable_id] = ds[ds.variable_id]*1000

    dsets.append(ds)



In [9]:
ds = xr.open_dataset(path)
ds

In [10]:
fig,ax = plt.subplot_mosaic([['a)','b)','c)'],['d)','e)','f)'],['g)','h)','i)']], 
                            subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(18,10), sharex=True, sharey=True)
cmap = mpl.cm.YlOrBr
norm = mpl.colors.LogNorm(vmin=1e-1, vmax=1e2)
for k,ds, sids in zip(ax.keys(), dsets, budgets.keys()):
    ds[ds.variable_id].plot.pcolormesh(ax=ax[k],cmap=cmap,norm=norm, add_colorbar=False)
    ax[k].coastlines()
    global_map(ax[k])
    ax[k].set_title('{} [Tg/year]    {}'.format(np.round(budgets[sids].values),ds.source_id))
    
cax = fig.add_axes([0.94,0.2,0.02,0.62])
fig.colorbar(mpl.cm.ScalarMappable(norm, cmap=cmap), cax=cax, extend='neither',
             label='{} [g m-2 year-1]'.format(ds.variable_id))
plt.savefig(snakemake.output.outpath, bbox_inches='tight', facecolor='white')